In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import plotly.io as pio
pio.renderers.default='notebook'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression, BayesianRidge
import time
import datetime
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from itertools import combinations
from sklearn.pipeline import Pipeline

In [2]:
#!pip install category_encoders

In [3]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [4]:
tripdata.head()

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female
3,144929,1.02,ss_3183,Exchange Place,40.716247,-74.033459,es_3183,Exchange Place,40.716247,-74.033459,...,Clear,2018-07-16,1.783112e-12,1985,33,working_day,weekday,July,summer,male
4,326659,1.02,others,Jersey & 3rd,40.723332,-74.045953,es_3211,Newark Ave,40.721525,-74.046305,...,cloudy_rain,2018-11-02,1.595320e-01,1987,31,working_day,weekday,November,autumn,male


In [5]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [6]:
df = tripdata[selected_features]

In [7]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [8]:
def getSampleSize(df, perc):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li  


def induceMissingValues(df , cols_list, perc, name ):
    random.seed(100)
    index = getAnIndex(df.index)
    sample_size = getSampleSize(df, perc)
    print(f"number of missing values produced {sample_size} for the {name}")
    for col in cols_list:
        idx = random.sample(index, sample_size)
        for i in idx:
             df.loc[i,col] = np.NaN
    return df

In [9]:
def ampute_each_variables(col_list, X_train, X_test):
    X_train = induceMissingValues(X_train, col_list, perc=10, name="train")
    print('Train set after Amputation', X_train.shape)
    X_test = induceMissingValues(X_test, col_list, perc=10, name="test")
    print('Test set after Amputation', X_test.shape)
    return X_train , X_test 

In [10]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [11]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [12]:
def run_multiple_imputation(X_train_amputed, X_test_amputed, y_train, y_test, random_state=0):
    print(f"run_multiple_imputation iteration {random_state} started" ,  datetime.datetime.now())
    impute_estimator = BayesianRidge()
    imputer = IterativeImputer(
        estimator=impute_estimator, max_iter=5, sample_posterior=False, random_state=random_state, tol=1e-1
    )
    ## Fit and tranform the missing values
    X_train_imputed = imputer.fit_transform(X_train_amputed)
    X_test_imputed = imputer.fit_transform(X_test_amputed)

    predictors = [var for var in X_train_amputed.columns]
    
    ## adjust the columns names to the imputed cols
    X_train_imputed = pd.DataFrame(X_train_imputed)
    X_test_imputed = pd.DataFrame(X_test_imputed)
    
    ## Run Linear regression on the 
    model_reg = model_evaluation(X_train_imputed,y_train)
    y_predict = model_reg.predict(X_test_imputed)
    
    print(f"run_multiple_imputation iteration {random_state} stopped" ,  datetime.datetime.now())
    return y_predict

### Multiple Imputation simulation with 2 variables

In [13]:
results ={}
col_list = [ ['dist','birthyear'],['start_lat','start_lon'],['end_lat','end_lon'], ['hour','temp'], 
            ['feelslike', 'dew'], ['snowdepth','winddir'],['sealevelpressure','visibility'],
             ['solarradiation','dist']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}
    

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:27:46.574708
2023-03-30 13:28:40.089208
2023-03-30 13:28:40.571077
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:28:40.583078
results are saving for the columns ['dist', 'birthyear']  =  2.3472947841507574 0.47899304244361474
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:28:53.359368
2023-03-30 13:29:39.424095
2023-03-30 13:29:39.932164
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:29:39.943388
results are saving

In [14]:
results

{('dist', 'birthyear'): {'rmse': 2.3472947841507574,
  'r2': 0.47899304244361474},
 ('start_lat', 'start_lon'): {'rmse': 2.2361619915255795,
  'r2': 0.5271593798176698},
 ('end_lat', 'end_lon'): {'rmse': 2.2360295045832985,
  'r2': 0.5272154073795429},
 ('hour', 'temp'): {'rmse': 2.236286008550537, 'r2': 0.5271069311012488},
 ('feelslike', 'dew'): {'rmse': 2.2360308656446053, 'r2': 0.5272148318155709},
 ('snowdepth', 'winddir'): {'rmse': 2.2359687264066372,
  'r2': 0.5272411088248119},
 ('sealevelpressure', 'visibility'): {'rmse': 2.2359873937324277,
  'r2': 0.5272332149925085},
 ('solarradiation', 'dist'): {'rmse': 2.331965175671093,
  'r2': 0.4857759592739128}}

### Multivariate simulation with 3 variables

In [15]:
results ={}
col_list = [ ['dist','birthyear','start_lat'],
            ['start_lat','start_lon','end_lat'],
            ['end_lat','end_lon','hour'], 
            ['hour','temp','feelslike'],
            ['feelslike', 'dew','snowdepth'],
            ['snowdepth','winddir','sealevelpressure'],
            ['sealevelpressure','visibility','solarradiation'],
            ['solarradiation','dist','birthyear']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:36:46.769176
2023-03-30 13:37:32.190411
2023-03-30 13:37:32.797067
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:37:32.808557
results are saving for the columns ['dist', 'birthyear', 'start_lat']  =  2.347523677988579 0.47889142666243967
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:37:55.960810
2023-03-30 13:38:40.171260
2023-03-30 13:38:40.769906
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:38:40.782423
result

In [16]:
results

{('dist', 'birthyear', 'start_lat'): {'rmse': 2.347523677988579,
  'r2': 0.47889142666243967},
 ('start_lat', 'start_lon', 'end_lat'): {'rmse': 2.2362288929389247,
  'r2': 0.527131086537393},
 ('end_lat', 'end_lon', 'hour'): {'rmse': 2.2363989910853337,
  'r2': 0.5270591465092247},
 ('hour', 'temp', 'feelslike'): {'rmse': 2.236370222426979,
  'r2': 0.5270713140932546},
 ('feelslike', 'dew', 'snowdepth'): {'rmse': 2.236056678903734,
  'r2': 0.5272039158692039},
 ('snowdepth', 'winddir', 'sealevelpressure'): {'rmse': 2.2359849420439377,
  'r2': 0.527234251739185},
 ('sealevelpressure',
  'visibility',
  'solarradiation'): {'rmse': 2.2359810498065604, 'r2': 0.5272358976491194},
 ('solarradiation', 'dist', 'birthyear'): {'rmse': 2.332221385642983,
  'r2': 0.4856629588175101}}

### Multivariate simulation with 4 variables

In [17]:
results ={}
col_list = [['dist','birthyear','start_lat','start_lon'],
            ['start_lat','start_lon','end_lat','end_lon'],
            ['end_lat','end_lon','hour','temp'], 
            ['hour','temp','feelslike','dew'],
            ['feelslike', 'dew','snowdepth','winddir'],
            ['snowdepth','winddir','sealevelpressure','visibility'],
            ['sealevelpressure','visibility','solarradiation','dist'],
            ['solarradiation','dist','birthyear','start_lat']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:47:10.739183
2023-03-30 13:47:45.359236
2023-03-30 13:47:45.753485
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:47:45.762492
results are saving for the columns ['dist', 'birthyear', 'start_lat', 'start_lon']  =  2.347714535348253 0.4788066893280605
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:48:01.989408
2023-03-30 13:48:43.120096
2023-03-30 13:48:43.627793
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:48:43.6

In [18]:
results

{('dist', 'birthyear', 'start_lat', 'start_lon'): {'rmse': 2.347714535348253,
  'r2': 0.4788066893280605},
 ('start_lat', 'start_lon', 'end_lat', 'end_lon'): {'rmse': 2.236451362907109,
  'r2': 0.5270369956638661},
 ('end_lat', 'end_lon', 'hour', 'temp'): {'rmse': 2.236439910555162,
  'r2': 0.5270418395205266},
 ('hour', 'temp', 'feelslike', 'dew'): {'rmse': 2.236439349405049,
  'r2': 0.5270420768624651},
 ('feelslike', 'dew', 'snowdepth', 'winddir'): {'rmse': 2.2360455646657216,
  'r2': 0.5272086158888418},
 ('snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility'): {'rmse': 2.2359956121070055, 'r2': 0.5272297396762535},
 ('sealevelpressure',
  'visibility',
  'solarradiation',
  'dist'): {'rmse': 2.3420814757071566, 'r2': 0.48130477056725784},
 ('solarradiation',
  'dist',
  'birthyear',
  'start_lat'): {'rmse': 2.3321963773693137, 'r2': 0.4856739891704608}}

### Multivaraite simulation with 5 variables 

In [19]:
results ={}
col_list = [['dist','birthyear','start_lat','start_lon','end_lat'],
            ['start_lat','start_lon','end_lat','end_lon','hour'],
            ['end_lat','end_lon','hour','temp','feelslike'],
            ['hour','temp','feelslike','dew','snowdepth'],
            ['feelslike', 'dew','snowdepth','winddir','sealevelpressure'],
            ['snowdepth','winddir','sealevelpressure','visibility','solarradiation'],
            ['sealevelpressure','visibility','solarradiation','dist','birthyear'],
            ['solarradiation','dist','birthyear','start_lat','start_lon']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:57:59.899519
2023-03-30 13:58:29.583396
2023-03-30 13:58:29.981464
run_multiple_imputation iteration 1200 stopped 2023-03-30 13:58:29.994500
results are saving for the columns ['dist', 'birthyear', 'start_lat', 'start_lon', 'end_lat']  =  2.3474923119068474 0.4789053519958104
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 13:58:47.309483
2023-03-30 13:59:26.943559
2023-03-30 13:59:27.448099
run_multiple_imputation iteration 1200 stopped 2023-03-3

In [20]:
results

{('dist',
  'birthyear',
  'start_lat',
  'start_lon',
  'end_lat'): {'rmse': 2.3474923119068474, 'r2': 0.4789053519958104},
 ('start_lat',
  'start_lon',
  'end_lat',
  'end_lon',
  'hour'): {'rmse': 2.236917136832374, 'r2': 0.5268399721305972},
 ('end_lat',
  'end_lon',
  'hour',
  'temp',
  'feelslike'): {'rmse': 2.236566705477668, 'r2': 0.526988209290637},
 ('hour', 'temp', 'feelslike', 'dew', 'snowdepth'): {'rmse': 2.236582361577262,
  'r2': 0.5269815870462784},
 ('feelslike',
  'dew',
  'snowdepth',
  'winddir',
  'sealevelpressure'): {'rmse': 2.2360352360175195, 'r2': 0.5272129836751501},
 ('snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility',
  'solarradiation'): {'rmse': 2.2361634289851264, 'r2': 0.5271587719104747},
 ('sealevelpressure',
  'visibility',
  'solarradiation',
  'dist',
  'birthyear'): {'rmse': 2.3424195135851713, 'r2': 0.48115503086774525},
 ('solarradiation',
  'dist',
  'birthyear',
  'start_lat',
  'start_lon'): {'rmse': 2.3327281346824607, 'r2': 0.

### Multivaraite simulation with 6 variables 

In [21]:
results ={}
col_list = [['dist','birthyear','start_lat','start_lon','end_lat','end_lon'],
            ['end_lat','end_lon','hour','temp','feelslike','dew'],
            ['hour','temp','feelslike','dew','snowdepth','winddir'],
            ['feelslike', 'dew','snowdepth','winddir','sealevelpressure','visibility'],
            ['snowdepth','winddir','sealevelpressure','visibility','solarradiation','dist'],
            ['sealevelpressure','visibility','solarradiation','dist','birthyear','start_lat'],
            ['solarradiation','dist','birthyear','start_lat','start_lon','end_lat']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 14:08:11.289258
2023-03-30 14:08:35.847798
2023-03-30 14:08:36.156162
run_multiple_imputation iteration 1200 stopped 2023-03-30 14:08:36.162165
results are saving for the columns ['dist', 'birthyear', 'start_lat', 'start_lon', 'end_lat', 'end_lon']  =  2.3477172716235932 0.4788054744195076
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 14:08:50.651275
2023-03-30 14:09:14.983774
2023-03-30 14:09:15.256288
run_multiple_imputation iteration 1200 stoppe

In [22]:
results

{('dist',
  'birthyear',
  'start_lat',
  'start_lon',
  'end_lat',
  'end_lon'): {'rmse': 2.3477172716235932, 'r2': 0.4788054744195076},
 ('end_lat',
  'end_lon',
  'hour',
  'temp',
  'feelslike',
  'dew'): {'rmse': 2.2366009122318227, 'r2': 0.526973740397999},
 ('hour',
  'temp',
  'feelslike',
  'dew',
  'snowdepth',
  'winddir'): {'rmse': 2.2366173860929224, 'r2': 0.5269667721476036},
 ('feelslike',
  'dew',
  'snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility'): {'rmse': 2.2360402764482155, 'r2': 0.5272108521766503},
 ('snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility',
  'solarradiation',
  'dist'): {'rmse': 2.3432291276953294, 'r2': 0.48079631049397686},
 ('sealevelpressure',
  'visibility',
  'solarradiation',
  'dist',
  'birthyear',
  'start_lat'): {'rmse': 2.342550053479108, 'r2': 0.4810972001895961},
 ('solarradiation',
  'dist',
  'birthyear',
  'start_lat',
  'start_lon',
  'end_lat'): {'rmse': 2.332884132262952, 'r2': 0.4853705992920745}}

### Multivaraite simulation with 7 variables 

In [23]:
results ={}
col_list = [['dist','birthyear','start_lat','start_lon','end_lat','end_lat','end_lon'],
            ['end_lat','end_lon','hour','temp','feelslike','dew','snowdepth'],
            ['hour','temp','feelslike','dew','snowdepth','winddir','sealevelpressure'],
            ['feelslike', 'dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation'],
            ['snowdepth','winddir','sealevelpressure','visibility','solarradiation','dist','birthyear'],
            ['sealevelpressure','visibility','solarradiation','dist','birthyear','start_lat','start_lon'],
            ['solarradiation','dist','birthyear','start_lat','start_lon','end_lat','end_lat']]
for cols in col_list:
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
    X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
    X_train_encoded = pd.DataFrame(X_train_encoded)
    X_test_encoded = pd.DataFrame(X_test_encoded)
    X_train_data = X_train_encoded.copy()
    X_test_data = X_test_encoded.copy()
    y_predict = run_multiple_imputation(
        X_train_data, X_test_data, y_train, y_test, random_state=1200
    )
    rmse = np.sqrt(mean_squared_error(y_test, y_predict))
    r2 = r2_score(y_test, y_predict)
    print("results are saving for the columns", cols, " = ", rmse, r2)
    results[tuple(cols)] = {"rmse": rmse, "r2": r2}

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 14:14:36.245145
2023-03-30 14:14:59.938396
2023-03-30 14:15:00.263224
run_multiple_imputation iteration 1200 stopped 2023-03-30 14:15:00.271217
results are saving for the columns ['dist', 'birthyear', 'start_lat', 'start_lon', 'end_lat', 'end_lat', 'end_lon']  =  2.347848963929505 0.4787470012508729
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 14:15:16.782331
2023-03-30 14:15:44.432042
2023-03-30 14:15:44.710932
run_multiple_imputation iteration 1

In [24]:
results

{('dist',
  'birthyear',
  'start_lat',
  'start_lon',
  'end_lat',
  'end_lat',
  'end_lon'): {'rmse': 2.347848963929505, 'r2': 0.4787470012508729},
 ('end_lat',
  'end_lon',
  'hour',
  'temp',
  'feelslike',
  'dew',
  'snowdepth'): {'rmse': 2.2365833811686033, 'r2': 0.5269811557761439},
 ('hour',
  'temp',
  'feelslike',
  'dew',
  'snowdepth',
  'winddir',
  'sealevelpressure'): {'rmse': 2.236607570586635, 'r2': 0.5269709239980389},
 ('feelslike',
  'dew',
  'snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility',
  'solarradiation'): {'rmse': 2.2361662457929308, 'r2': 0.5271575806704399},
 ('snowdepth',
  'winddir',
  'sealevelpressure',
  'visibility',
  'solarradiation',
  'dist',
  'birthyear'): {'rmse': 2.3434896349859393, 'r2': 0.48068085967155716},
 ('sealevelpressure',
  'visibility',
  'solarradiation',
  'dist',
  'birthyear',
  'start_lat',
  'start_lon'): {'rmse': 2.3430552544203525, 'r2': 0.4808733599770785},
 ('solarradiation',
  'dist',
  'birthyear',
  'star

### with all the variables

In [25]:
cols = ['start_lat','start_lon','end_lat',
                     'end_lon','hour','temp', 'feelslike', 'dew','snowdepth',
                     'winddir','sealevelpressure','visibility','solarradiation','dist','birthyear']

X_train, X_test, y_train, y_test = split_dataset(df)
X_ampute_train , X_ampute_test  = ampute_each_variables(cols, X_train, X_test)
X_train_encoded , X_test_encoded =  one_hot_encoding(X_ampute_train, X_ampute_test)
X_train_encoded = pd.DataFrame(X_train_encoded)
X_test_encoded = pd.DataFrame(X_test_encoded)
X_train_data = X_train_encoded.copy()
X_test_data = X_test_encoded.copy()
y_predict = run_multiple_imputation(
    X_train_data, X_test_data, y_train, y_test, random_state=1200
)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
r2 = r2_score(y_test, y_predict)


Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
number of missing values produced 25235 for the train
Train set after Amputation (252346, 21)
number of missing values produced 6309 for the test
Test set after Amputation (63087, 21)
run_multiple_imputation iteration 1200 started 2023-03-30 14:22:21.647937
2023-03-30 14:23:55.063647
2023-03-30 14:23:55.474804
run_multiple_imputation iteration 1200 stopped 2023-03-30 14:23:55.490390


In [26]:
print("results are saving for the columns", cols, " = ", rmse, r2)   

results are saving for the columns ['start_lat', 'start_lon', 'end_lat', 'end_lon', 'hour', 'temp', 'feelslike', 'dew', 'snowdepth', 'winddir', 'sealevelpressure', 'visibility', 'solarradiation', 'dist', 'birthyear']  =  2.3427708183302 0.48099939148144233
